In [2]:

def my_agent(obs, config):
    import numpy as np
    import random
    printSig=0
    def visionDepth1(test_moves,obs,config,invalid_moves):
        piece=obs.mark
        if piece==1:
            oponent_piece=2
        else:oponent_piece=1
        grid = np.asarray(obs.board).reshape(config.rows, config.columns)
        bad_vision=[]
        obs_next=obs
        for i,_ in test_moves.items():
            obs_next.board=np.asanyarray(drop_piece(grid, i, obs.mark, config)).reshape(config.rows*config.columns,1)
            op_moves = [col for col in range(config.columns) if obs_next.board[col] == 0]
            for j in op_moves:
                if check_winning_move(obs_next,config,j,oponent_piece)==True:
                    bad_vision.append(i)
                    break
                    
        for k in bad_vision:
            invalid_moves.add(k)
            test_moves[k]=-1
        return test_moves,invalid_moves
    
    # Gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, piece, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = piece
        return next_grid

    # Returns True if dropping piece in column results in game win
    def check_winning_move(obs, config, col, piece):
        # Convert the board to a 2D grid
        grid = np.asarray(obs.board).reshape(config.rows, config.columns)
        next_grid = drop_piece(grid, col, piece, config)
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[row,col:col+config.inarow])
                if window.count(piece) == config.inarow:
                    return True
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(next_grid[row:row+config.inarow,col])
                if window.count(piece) == config.inarow:
                    return True
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if window.count(piece) == config.inarow:
                    return True
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if window.count(piece) == config.inarow:
                    return True
        return False
    def check_semi_winning_move(obs, config, col, piece):
        # Convert the board to a 2D grid
        piece=obs.mark
        if piece==1:
            oponent_piece=2
        else:oponent_piece=1
        grid = np.asarray(obs.board).reshape(config.rows, config.columns)
        next_grid = drop_piece(grid, col, piece, config)
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[row,col:col+config.inarow])
                if (window.count(piece)+1 == config.inarow):
                    if printSig:print("semi indicated, hori",row,col)
                    if window.count(oponent_piece)==0:return True
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(next_grid[row:row+config.inarow,col])
                if (window.count(piece)+1 == config.inarow):
                    if printSig:print("semi indicated, vertical",row,col)
                    if window.count(oponent_piece)==0:return True
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if (window.count(piece)+1 == config.inarow):
                    if printSig:print("semi indicated, pos dialg",row,col)
                    if window.count(oponent_piece)==0:return True
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(next_grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if (window.count(piece)+1 == config.inarow):
                    if printSig:print("semi indicated, neg dialg",row,col)
                    if window.count(oponent_piece)==0:return True
        return False

    valid_moves = {col:0 for col in range(config.columns) if obs.board[col] == 0}
    invalid_moves=set()
#     print(valid_moves)
# win move, Rank 1
#==========================================
    win_moves=[]
    for i in valid_moves:
        if check_winning_move(obs,config,i,obs.mark)==True:
            win_moves.append(i)
    if win_moves !=[]:
        send=random.choice(win_moves)
        if printSig:print(send)
        return send
#==========================================
# block move(anti lose move)
#==========================================
    block_moves=[]
    for i in valid_moves:
        if obs.mark==1:
            oponent=2
        else: oponent=1
        if check_winning_move(obs,config,i,oponent)==True:
            block_moves.append(i)
    # but a move that leads to failure is useless. remove them
    #==========================================    
#     block_moves,invalid_moves=visionDepth1(block_moves,obs,config,invalid_moves)
    #==========================================
    if block_moves !=[]:
        send=random.choice(block_moves)
        if printSig:print(send)
        return send
#==========================================
# semi block move(anti lose move)
#==========================================
    semi_block_moves=[]
    for i in valid_moves:
        if obs.mark==1:
            oponent=2
        else: oponent=1
        if check_semi_winning_move(obs,config,i,oponent)==True:
            semi_block_moves.append(i)

    # but a move that leads to failure is useless. remove them
    #==========================================    
#     semi_block_moves,invalid_moves=visionDepth1(semi_block_moves,obs,config,invalid_moves)
    #==========================================
    if semi_block_moves !=[]:
        send=random.choice(semi_block_moves)
#         for i in semi_block_moves:
#             valid_moves[i]+=1
        if printSig:print(send)
#==========================================

#Semi lose block, then semi win move add.
#==========================================
    semi_win_moves=[]
    for i in valid_moves:
        if check_semi_winning_move(obs,config,i,obs.mark)==True:
            semi_win_moves.append(i)
    #but still, 2 moves are required so shall not lead to lose move
#     semi_win_moves,invalid_moves=visionDepth1(semi_win_moves,obs,config,invalid_moves)
    if semi_win_moves !=[]:
        if printSig:print("semi win move",semi_win_moves)
    for i in semi_win_moves:
        valid_moves[i]+=1
        if printSig:print(send)
#==========================================

#     least_depth,invalid_moves=visionDepth1(least_depth,obs,config,invalid_moves)
#     for i in valid_moves:
#         if i in invalid_moves:
#             valid_moves[i]=-1
    print(valid_moves,invalid_moves)
    valid_moves,invalid_moves=visionDepth1(valid_moves,obs,config,invalid_moves)
    popularity=0
    final_blind=[]
    for i,j in valid_moves.items():
        if popularity<j:
            popularity=j
            final_blind=[i]
        elif popularity==j:
            final_blind.append(i)
    for i in final_blind:
        if i in invalid_moves:
            final_blind.remove(i)
    print("best randoms")
    print(final_blind)
    if final_blind==[]:
        final_blind=valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
        least_depth=[]#try to go with col with vacancy
    depth_max=0
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    least_depth=[]
    for i in final_blind:
#         colList=np.asarray([grid[_][i] for _ in range(config.rows)]).reshape(config.columns,1)
        colList=[grid[_][i] for _ in range(config.rows)]
        num_spaces=colList.count(0)
        if num_spaces>depth_max:
            depth_max=num_spaces
            least_depth=[i]
        else:
            least_depth.append(i)
    print("shallow randoms")    
    print(least_depth)    
    
    return random.choice(least_depth)

ModuleNotFoundError: No module named 'kaggle_environments'

Review  
Current method only passively blocks loosing cases.  
## It does not :  
- acively search for winning case  
- prevent opponent's random winning case(lack of depth)  
## There are some winning hints:  
- two consecutive parts
- 33s  

## I can try:  
Removing shallow search - this was for preventing dead end(lacking of column choice)  
however random(which wasn't working properly in prev. ver) is now alive, and does kinda prevent former problem(lack of col choice).  

# Strategic change  
it is too time consuming to add importance then list up. Try to import voting system.  
count up cols that show up in each method, then eliminate must-not moves.  
layer-y stack up? :   


priorty list :   
1. Win move  
2. Lose move(in single depth)  
3. Semi-lose move  
4. Semi-win move  
-1. shallowest move. Prevent from lack in choice   
722 conclusion :  
semi - winning is good. but semi blocking?? not so sure - add both version in submission and chose one.  
Next goal :  2 connection and semi win priorty, and 33 block



2
